In [3]:
import pandas as pd
import ipynb.fs.full.features as features_nltk

df_train = pd.read_csv('./data/train.csv')

In [4]:
def concatenar_categoricos(df, columna1, columna2, N=0, df_aux=None, devolver_cols=False):
    '''Concatena dos columnas categoricas y hace one hot'''
    '''Ver referencia columna_a_ohe para los ultimos 3 parametros'''
    
    def limpiar_espacios_blanco(texto):
        return texto.replace(' ', '_')
        
    df = df.copy()
    df[columna1] = df[columna1].apply(limpiar_espacios_blanco)
    df[columna2] = df[columna2].apply(limpiar_espacios_blanco)
    df[columna1 + '_' + columna2] = df[columna1] + '_' +df[columna2]
    
    features_nltk
    return features_nltk.columna_a_ohe(df, columna1 + '_' + columna2, N, df_aux, devolver_cols)

In [5]:
def cantidad_propiedad_misma_cantidad(df, agrupar_col, misma_cantidad_col):
    '''Cantidad de casas con la misma cantidad de banos que la actual'''
    '''Por ejemplo, retorna para cada propiedad, cuantas hay mas de su tipo con x banos ej.'''
    '''cantidad_propiedad_misma_cantidad(df, 'banos', 'tipodepropiedad'), para cada propiedad, se fija
    cuantas mas de su tipo con tantos banos hay'''
    '''Retorna el df con la columna agregada'''
    df_aux = df.copy()
    df_aux = df_aux.groupby([agrupar_col, misma_cantidad_col]).agg({'id': 'count'})
    df_aux = df_aux.rename(columns={'id':'cantidad'})
    df_aux = df_aux.reset_index()
    
    def get_cantidad(col1, col2):
        cantidad = df_aux[(df_aux[agrupar_col] == col1) & (df_aux[misma_cantidad_col] == col2)]['cantidad']
        return cantidad.values[0] if len(cantidad.values > 0) else 0
        
    df['cantidad_' + agrupar_col + '_' + misma_cantidad_col] = df.apply(lambda x: get_cantidad(x[agrupar_col], x[misma_cantidad_col]), axis=1)
    return df

In [6]:
def cantidad_atributo(df, col):
    '''Simil a value_counts. Dada una columna, agrega al df, cuantas propiedades existen con esa misma propiedad'''
    df_aux = df.copy()
    df_aux = df_aux.groupby([col]).agg({'id': 'count'})
    df_aux = df_aux.rename(columns={'id':'cantidad'})
    df_aux = df_aux.reset_index()

    def get_cantidad(col1):
        cantidad = df_aux[df_aux[col] == col1]['cantidad']
        return cantidad.values[0] if len(cantidad.values > 0) else 0
    
    df['cantidad_propiedades_en_' + col] = df.apply(lambda x: get_cantidad(x[col]), axis=1)
    return df

In [7]:
def ranking_atributo(df, col):
    '''Cuenta cuantas propiedades hay con los atributos de la columna recibida. Y para cada propiedad,
    se fija en que posicion esta del ranking. Por ejemplo, si hay dos propuedades, una con dos banos y otra 
    con uno solo, el rankin de las que tengan dos banos en 1, y de las propiedades que tengan un solo bano es 
    dos'''
    
    '''Rellena los nulos con "otros" '''
    df[col] = df[col].fillna('otros')
    
    df_aux = df.copy()
    df_aux = df_aux.groupby([col]).agg({'id': 'count'})
    df_aux = df_aux.rename(columns={'id':'cantidad'})
    df_aux = df_aux.reset_index()
    
    df_aux = df_aux.nlargest(df_aux.shape[0], 'cantidad') 
    lista_ranking = df_aux[col].tolist()
    rank = {}
    contador = 1
    
    for item in lista_ranking:
        rank[item] = contador
        contador = contador + 1

    def get_ranking(col1):
        item = df_aux[df_aux[col] == col1][col].values[0]
        return rank[item]
    
    df['ranking_en_' + col] = df.apply(lambda x: get_ranking(x[col]), axis=1)
    return df

In [13]:
df = df_train.head()
# df = df_train

#Ejs:
# concatenar_categoricos(df, 'tipodepropiedad', 'provincia')
# cantidad_propiedad_misma_cantidad(df, 'tipodepropiedad', 'provincia')
# cantidad_atributo(df, 'provincia')
# ranking_atributo(df, 'provincia')

def aplicar_features(df):
    df = concatenar_categoricos(df, 'tipodepropiedad', 'provincia')
    df = concatenar_categoricos(df, 'provincia', 'ciudad')
    # -- 
    display('Check-1')
    df = cantidad_atributo(df, 'provincia')
    df = cantidad_atributo(df, 'ciudad')
    df = cantidad_atributo(df, 'antiguedad')
    df = cantidad_atributo(df, 'habitaciones')
    df = cantidad_atributo(df, 'garages')
    df = cantidad_atributo(df, 'banos')
    df = cantidad_atributo(df, 'gimnasio')
    df = cantidad_atributo(df, 'usosmultiples')
    df = cantidad_atributo(df, 'escuelascercanas')
    df = cantidad_atributo(df, 'centroscomercialescercanos')
    df = cantidad_atributo(df, 'piscina')
    # --
    display('Check-2')
    df = ranking_atributo(df, 'provincia')
    df = ranking_atributo(df, 'tipodepropiedad')
    df = ranking_atributo(df, 'ciudad')
    df = ranking_atributo(df, 'antiguedad')
    df = ranking_atributo(df, 'habitaciones')
    df = ranking_atributo(df, 'garages')
    df = ranking_atributo(df, 'banos')
    df = ranking_atributo(df, 'gimasio')
    df = ranking_atributo(df, 'usosmultiples')
    df = ranking_atributo(df, 'escuelascercanas')
    df = ranking_atributo(df, 'centrocomercialescercanos')
    # --
    display('Check-3')
    df = cantidad_propiedad_misma_cantidad(df, 'tipodepropiedad', 'provincia')
    df = cantidad_propiedad_misma_cantidad(df, 'tipodepropiedad', 'ciudad')
    df = cantidad_propiedad_misma_cantidad(df, 'tipodepropiedad', 'antiguedad')
    df = cantidad_propiedad_misma_cantidad(df, 'tipodepropiedad', 'habitaciones')
    df = cantidad_propiedad_misma_cantidad(df, 'tipodepropiedad', 'garages')
    df = cantidad_propiedad_misma_cantidad(df, 'tipodepropiedad', 'banos')
    df = cantidad_propiedad_misma_cantidad(df, 'tipodepropiedad', 'gimnasio')
    df = cantidad_propiedad_misma_cantidad(df, 'tipodepropiedad', 'usosmultiples')
    df = cantidad_propiedad_misma_cantidad(df, 'tipodepropiedad', 'pisicina')
    display('Check-4')
    df = cantidad_propiedad_misma_cantidad(df, 'ciudad', 'antiguedad')
    df = cantidad_propiedad_misma_cantidad(df, 'ciudad', 'habitaciones')
    df = cantidad_propiedad_misma_cantidad(df, 'ciudad', 'garages')
    df = cantidad_propiedad_misma_cantidad(df, 'ciudad', 'banos')
    df = cantidad_propiedad_misma_cantidad(df, 'ciudad', 'gimnasio')
    df = cantidad_propiedad_misma_cantidad(df, 'ciudad', 'usosmultiples')
    df = cantidad_propiedad_misma_cantidad(df, 'ciudad', 'ciudad')
    display('Check-5')
    df = cantidad_propiedad_misma_cantidad(df, 'provincia', 'antiguedad')
    df = cantidad_propiedad_misma_cantidad(df, 'provincia', 'habitaciones')
    df = cantidad_propiedad_misma_cantidad(df, 'provincia', 'garages')
    df = cantidad_propiedad_misma_cantidad(df, 'provincia', 'banos')
    df = cantidad_propiedad_misma_cantidad(df, 'provincia', 'gimnasio')
    df = cantidad_propiedad_misma_cantidad(df, 'provincia', 'usosmultiples')
    df = cantidad_propiedad_misma_cantidad(df, 'provincia', 'pisina')

Index(['id', 'titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio'],
      dtype='object')

/home/tomas/Facultad/datos/datos-tp2/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/tomas/Facultad/datos/datos-tp2/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
